In [1]:
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Cursor

import json
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# from textblob import TextBlob

# import re
import os, wget

In [2]:
dataset_path = '../Dataset/'
file_user_image = 'user_image_downloaded.txt'
file_user_text = 'user_text.json'

In [3]:
images_failed_to_download = set()

In [4]:
print(len(images_failed_to_download))

0


In [5]:
def check_path(path):
    if not os.path.isdir(path):
        os.mkdir(path)

def check_file(file):
    if not os.path.isfile(file):
        open(file, 'w').close()

def download_files(__urls__, path = dataset_path+'Temporary'):
    check_path(path)

    print('downloading...')

    total = len(__urls__)
    print(total)
    for index, key in enumerate(__urls__):
        try:
            url = __urls__[key]
            
            file = url.split('/')[-1]
            extension = file.split('.')[-1]
            
#             if not extenstion in ['jpg','jpeg','png','bmp','svg']:
#                 continue
            
            print('\ndownloading ',index+1, 'out of', total)
            wget.download(url)

            os.rename(file,path + '/' + str(key) + '.' + extension)
        except:
            print('exception')
            images_failed_to_download.add(key)
    print('downloading complete')
    
def load_user_image_text():
    check_file(file_user_image)
    check_file(file_user_text)
    
    _user_image_downloaded = set()
    _user_text = dict()

    if os.stat(file_user_image).st_size != 0:
        _user_image_downloaded = set(open(file_user_image).read().split(','))

    if os.stat(file_user_text).st_size != 0:
        with open(file_user_text,'r') as file:
            _user_text = json.load(file)
        
    return _user_image_downloaded, _user_text

def download_save_images_tweets_from_single_user(userId):
    if(type(userId) != int):
        print('User Id is not int')
        return
    print('Downloading for : ',userId)
    _user_image_downloaded, _user_text = load_user_image_text()
    if str(userId) in _user_image_downloaded :
        print('Already downloaded')
        return
    
    __urls__, __texts__ = twitter_client.get_urls_text(userId)
    download_files(__urls__, dataset_path + 'mixed')
    
    _user_text = {**_user_text, **__texts__}
    _user_image_downloaded.add(str(id))
    
    with open(file_user_text, 'w') as file:
        json.dump(_user_text, file)
    file.close()
        
    file = open(file_user_image, 'w')
    for element in _user_image_downloaded:
        file.write(str(element))
        file.write(',')
    file.close()
    
    
def download_save_images_tweets_from_friends_list(friends):
    _user_image_downloaded, _user_text = load_user_image_text()
    # print('user image downloaded: ',user_image_downloaded)
    # print(user_text,'\n')
    for index, friend in enumerate(friends):

        print(index,'/',len(friends),'^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^')
        id = friend.id
        if str(id) in _user_image_downloaded :
            continue

        __urls__, __texts__ = twitter_client.get_urls_text(id)
        download_files(__urls__,dataset_path + 'mixed')

        _user_text = {**_user_text, **__texts__}
        _user_image_downloaded.add(str(id))

        with open(file_user_text,'w') as file:
            json.dump(_user_text, file)
        file.close()

        file = open(file_user_image, 'w')
        for element in _user_image_downloaded:
            file.write(str(element))
            file.write(',')
        file.close()

In [6]:
class TwitterAuthenticator():
    '''
    Twitter Authentication
    '''
    def authenticate_twitter_app(self):
        with open('credentials.json','r') as file:
            credentials = json.load(file)

        auth = OAuthHandler(credentials['CONSUMER_KEY'], credentials['CONSUMER_SECRET'])
        auth.set_access_token(credentials['ACCESS_TOKEN'], credentials['ACCESS_SECRET'])
        return auth

In [7]:
class TwitterClient():

    def __init__(self,twitter_user = None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.api = API(self.auth)
        self.twitter_user = twitter_user
        
    def get_urls_text(self,id):
        tweets =  []
        image_urls = dict()
        text = dict()
        
        for tweet in Cursor(self.api.user_timeline, id = id, tweet_mode = 'extended').items():
            media = tweet.entities.get('media',[])
            if media == None or len(media) == 0:
                continue
            url = media[0]['media_url']
            
            print(url)

            extension = url.split('.')[-1]
            if extension != 'jpg' and extension != 'jpeg' and extension != 'png' and extension != 'svg' and extenion != 'bmp':
                continue

            if tweet.full_text is None or len(tweet.full_text) is 0:
                text[tweet.id] = tweet.text
            else:
                text[tweet.id] = tweet.full_text
                
            image_urls[tweet.id] = url
            
        return image_urls, text

    def get_api(self):
        return self.api

    def get_user_timeline_tweets(self, count):
        tweets = []
        for tweet in Cursor(self.api.user_timeline, id = self.twitter_user,).items(count):
            tweets.append(tweet)
        return tweets

    def get_friend_list(self,user_id = None, count = 30):
        friend_list = []
        for friend in Cursor(self.api.friends, id = user_id).items(count):
            friend_list.append(friend)
        return friend_list

    def get_home_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.api.home_timeline, id = self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets

In [8]:
twitter_client = TwitterClient()

In [ ]:
download_save_images_tweets_from_single_user(12382182)

http://pbs.twimg.com/media/D4hVpF2W0AAJUfD.jpg
http://pbs.twimg.com/media/D4hvnc-W4AEJz9S.jpg
http://pbs.twimg.com/media/DlRaP6lUcAA742F.jpg
http://pbs.twimg.com/media/D4R89VaWAAA2Prb.jpg
http://pbs.twimg.com/media/D3SApYvUEAEVDLU.jpg
http://pbs.twimg.com/media/D3OXVnJW0AEr4s_.jpg
http://pbs.twimg.com/media/D3Jiq-LX4AAw3Zc.jpg
http://pbs.twimg.com/media/D3Epg96WkAErN_M.jpg
http://pbs.twimg.com/media/D3ET2HTXgAMh5wO.jpg
http://pbs.twimg.com/media/D3DGP_FUkAAcWl-.jpg
http://pbs.twimg.com/media/D2ySJ0nUkAEU7my.jpg
http://pbs.twimg.com/media/D2wn1ykWoAAyyJh.jpg
http://pbs.twimg.com/media/D2wWiJ7X0AAy6Bs.jpg
http://pbs.twimg.com/media/D2v4IbPWsAEj4OH.jpg
http://pbs.twimg.com/media/D2hKj4tX4AEyv5x.jpg
http://pbs.twimg.com/tweet_video_thumb/D19kYuDWkAIAhmV.jpg
http://pbs.twimg.com/media/D189_F3WsAIcjyC.jpg
http://pbs.twimg.com/media/D189-gDWsAA_NuL.jpg
http://pbs.twimg.com/media/D18993xW0AALqLv.jpg
http://pbs.twimg.com/media/D14vv_KW0AIyxlU.jpg
http://pbs.twimg.com/media/D14vh-WWwAEEnTa.jpg
h

http://pbs.twimg.com/media/DYq01AnU0AAwxjp.jpg
http://pbs.twimg.com/media/DOpjBccXcAAIRpv.jpg
http://pbs.twimg.com/media/DOpgqsgXkAIrf-6.jpg
http://pbs.twimg.com/media/DOpgOvZXkAI3C2K.jpg
http://pbs.twimg.com/media/DYimQ4wU8AAAnYp.jpg
http://pbs.twimg.com/media/DYikCK_V4AASyPm.jpg
http://pbs.twimg.com/media/DYijo9oV4AAriFi.jpg
http://pbs.twimg.com/media/DYic1OEVwAA-7Fs.jpg
http://pbs.twimg.com/media/DYibCLcVMAAtQKG.jpg
http://pbs.twimg.com/media/DYiWOzSUQAAX-TE.jpg
http://pbs.twimg.com/media/DYiYQYHU8AADVcq.jpg
http://pbs.twimg.com/media/DYiXmZSU8AATJtF.jpg
http://pbs.twimg.com/media/DYcnLMIU8AA-GEr.jpg
http://pbs.twimg.com/media/DYT3_kRVMAAPp50.jpg
http://pbs.twimg.com/media/DYXHW1kV4AAHrFO.jpg
http://pbs.twimg.com/media/DYSCQGPW4AExeSD.jpg
http://pbs.twimg.com/media/DYT6hTuVwAAG6S_.jpg
http://pbs.twimg.com/media/DYRPoG_WsAoX1gJ.jpg
http://pbs.twimg.com/media/DYKpgDJWsAA7gD1.jpg
http://pbs.twimg.com/media/DYLdKjlWsAMMh-6.jpg
http://pbs.twimg.com/media/DXznkxMVoAAr77T.jpg
http://pbs.tw

http://pbs.twimg.com/media/DGbU57UXoAAd22X.jpg
http://pbs.twimg.com/tweet_video_thumb/DGbVVNtW0AA9NHv.jpg
http://pbs.twimg.com/media/Af1em46CMAAp_F-.jpg
http://pbs.twimg.com/media/DGTvhvJU0AAJzqj.jpg
http://pbs.twimg.com/media/DGRtIHIVoAADsbM.jpg
http://pbs.twimg.com/tweet_video_thumb/DGPSGMwW0AE0tJu.jpg
http://pbs.twimg.com/media/DGNilokVwAA0heQ.jpg
http://pbs.twimg.com/media/DGLyM8LXkAEOYMZ.jpg
http://pbs.twimg.com/media/DGK4wrZVoAAP71S.jpg
http://pbs.twimg.com/media/DFz-rllXkAEAFm6.jpg
http://pbs.twimg.com/media/DFzRXTfUQAAGJN9.jpg
http://pbs.twimg.com/tweet_video_thumb/DFxykOlUQAAQ1t2.jpg
http://pbs.twimg.com/media/DFvknBJXcAQfh4Q.jpg
http://pbs.twimg.com/media/DFu143lUMAEccXh.jpg
http://pbs.twimg.com/media/DFsxu0fXcAQjK0x.jpg
http://pbs.twimg.com/media/DFsC37DXgAETYxo.jpg
http://pbs.twimg.com/media/DFlkAoyXkAAzfB0.jpg
http://pbs.twimg.com/media/DFhroKBWAAEFOo7.jpg
http://pbs.twimg.com/media/DFblre_WAAURZzt.jpg
http://pbs.twimg.com/media/DFNT04iW0AIZPJE.jpg
http://pbs.twimg.com/med


downloading  18 out of 503

downloading  19 out of 503

downloading  20 out of 503

downloading  21 out of 503

downloading  22 out of 503

downloading  23 out of 503

downloading  24 out of 503

downloading  25 out of 503

downloading  26 out of 503

downloading  27 out of 503

downloading  28 out of 503

downloading  29 out of 503

downloading  30 out of 503

downloading  31 out of 503

downloading  32 out of 503

downloading  33 out of 503

downloading  34 out of 503

downloading  35 out of 503

downloading  36 out of 503

downloading  37 out of 503

downloading  38 out of 503

downloading  39 out of 503

downloading  40 out of 503

downloading  41 out of 503

downloading  42 out of 503

downloading  43 out of 503

downloading  44 out of 503

downloading  45 out of 503

downloading  46 out of 503

downloading  47 out of 503

downloading  48 out of 503

downloading  49 out of 503

downloading  50 out of 503

downloading  51 out of 503

downloading  52 out of 503

downloading  53 out

In [11]:
friends = twitter_client.get_friend_list(user_id = 'natasharekhtman',count = 30)

In [ ]:
download_save_images_tweets_from_friends_list(friends)

In [10]:
user_image_downloaded, user_text = load_user_image_text()

In [15]:
print(len(friends))

30


In [16]:
for friend in friends:
    print(friend.id, end=',')

745001021714272257,24725678,748508759941586944,533462148,361657897,2316388819,3856634122,2854305398,409486555,813286,50393960,170877963,2374778886,195054982,258147572,100102221,3367146257,2507601566,3337373759,365203100,911946993093799936,992940835196014593,1375542750,525554063,12382182,16156716,3115701824,20786898,1042472187184205826,1027647597304049665,

In [13]:
print(user_image_downloaded)

{'823529382954336258', '1071745183', '1430797741', '1013817930', '3345645321', '841360138749513729', '26601124', '74171204', '4528949564', '171921202', '107354991', '1017375038', '1023727602', '886918277397495808\n', '996927835', '1613469289', '517885104', '2802675478', '3041039242', '2459131656', '31663006', '838088228653707264', '732666770637492225', '70273076'}


In [14]:
print(user_text)

{'933933303199158272': '@Aviralmathur4 Agree... https://t.co/hIQxoGLcsN', '866586544202805250': 'RT @Kcl33172101: https://t.co/5s9VX3tVHo', '865520546301136897': "Paul Kalanithi's book - When Breath Becomes Air https://t.co/1Egs3ifsUb https://t.co/8SZuGVGrD7", '735530514493378560': 'RT @kishorerubens: Any info u want abt medical profession contact @RxPG its amazing @iamMNK @ravichandrasek6 https://t.co/gHj984XOzw', '417124072456060928': 'RT @Dark_Ryder: @RxPG http://t.co/TkB5x9BTVs', '358241889885356032': "RT @drswagat17: @RxPG you can't reach your destination without going through rxpg. .. http://t.co/74bQYgkNCy", '1120690960894242817': '@ariella8 @yro854 @smlungpathguy @lara_pijuan @SansanoValero @raghupillappa @tralimiguel @TeamCaptainJohn @kis_lorand 1⃣Great discussion! Agree – DDx cribriform adeno vs LCNEC. I WOULD get NE markers. Necrosis, palisading, &amp; some cribriform spaces have rigid borders (👇 from https://t.co/jxBmKn10GS). Regardless - adeno-type molecular testing needed

In [17]:
for single_user_text in user_text:
    print(single_user_text)

In [17]:
for friend in friends:
    if str(friend.id) in user_image_downloaded:
        print(',',end = ' ')
    else:
        print(friend.id)

745001021714272257
24725678
748508759941586944
533462148
361657897
2316388819
3856634122
2854305398
409486555
813286
50393960
170877963
2374778886
195054982
258147572
100102221
3367146257
2507601566
3337373759
365203100
911946993093799936
992940835196014593
1375542750
525554063
12382182
16156716
3115701824
20786898
1042472187184205826
1027647597304049665


In [12]:
#no use
__urls, __text = twitter_client.get_urls_text(20051580)
print(len(__urls))

download_files(__urls)

http://pbs.twimg.com/media/DPX_Y3yXUAAhBAu.jpg
http://pbs.twimg.com/media/DANaD96UAAUOtmT.jpg
http://pbs.twimg.com/media/DALyLqoV0AAZO64.jpg
http://pbs.twimg.com/tweet_video_thumb/CjCSApHUgAEwxGJ.jpg
http://pbs.twimg.com/media/Bcdrm7bCQAAwLb0.jpg
http://pbs.twimg.com/media/BPi6A62CEAAv3zr.jpg
6
downloading...
6

downloading  1 out of 6

downloading  2 out of 6

downloading  3 out of 6

downloading  4 out of 6

downloading  5 out of 6

downloading  6 out of 6
downloading complete


In [14]:
for index, key in enumerate(__urls):
    print(index, key, __urls[key])

0 933933303199158272 http://pbs.twimg.com/media/DPX_Y3yXUAAhBAu.jpg
1 866586544202805250 http://pbs.twimg.com/media/DANaD96UAAUOtmT.jpg
2 865520546301136897 http://pbs.twimg.com/media/DALyLqoV0AAZO64.jpg
3 735530514493378560 http://pbs.twimg.com/tweet_video_thumb/CjCSApHUgAEwxGJ.jpg
4 417124072456060928 http://pbs.twimg.com/media/Bcdrm7bCQAAwLb0.jpg
5 358241889885356032 http://pbs.twimg.com/media/BPi6A62CEAAv3zr.jpg


In [32]:
def get_files(loc_data):
#     rex = re.compile(r'^.*\.({})$'.format('jpg|jpeg|bmp|png'), re.I)
    files = set(os.path.join(loc_data,base).split('/')[-1].split('.')[0] for base in os.listdir(loc_data))
#     print(files)
    return files

def getPathologyName():
    pathology= get_files('../Dataset/pathology')
    other = get_files('../Dataset/other')
    print(len(pathology),len(other), len(pathology) + len(other))
    mixed = set(single_text for single_text in user_text)
    print(len(mixed))
    return pathology,other,mixed

In [25]:
images = get_files('../Dataset/mixed')
print(len(images))

In [30]:
pathology, other, mixed = getPathologyName()

4732 1995 6727
0


In [86]:
temp = mixed - other
print(len(temp))

8919


In [31]:
temp = dict()
for id_ in pathology:
    temp[id_] = user_text[id_]

print(len(temp))

1073201052063805440


In [92]:
with open("pathology.json",'w') as file:
        json.dump(temp, file)
file.close()